# Бинарная классификация? Бинарная кросс-энтропия!

Мы коснулись задачи регрессии (аппроксимация скрытой зависимости), в которой нейронная сеть подбирает параметры активационных функций (этими параметрами являются веса и смещения скрытого слоя) и ищет оптимальную для аппроксимации линейную комбинацию этих активационных функций (веса выходного линейного слоя).

**Задача бинарной классификации**

Пусть у нас есть два класса. Мы хотим, чтобы сеть выдавала вероятность отнесения входа к одному из классов. Для этого вещественный выход линейного слоя дополнительно пропускается через сигмоиду и полученное значение интерпретируется как вероятность.

\*) Для задачи бинарной классификации можно обойтись и без выходной сигмоиды; но тогда сеть будет дольше обучаться, т.к. придется ориентироваться в пространстве решений бесконечного размера, а не в интервале [0...1].

- выходная функция активации - сигмоида:

$$\sigma(y) = \frac{1}{1 + e^{-y}} \\
\sigma'(y) =  \sigma(y) (1 - \sigma(y))$$

- функция потерь:
  - **почему не подходит MSE**? 
    - $MSE_i = (\sigma(y_i) - t_i)^2$, где $t_i \in \{0,1\}$ - это таргет (ground truth)
    - $MSE_i' = 2 (\sigma(y_i) - t_i) \sigma' = 2 (\sigma(y_i) - t_i) \sigma(y_i)(1  - \sigma(y_i))$
      - потому что производная такой функции потерь "слишком легко" обращается в 0 (происходит паралич сети): три сомножителя, только один из которых зависит от таргета, остальные могут зануляться и обращать все в 0 (при малых значениях выходов $y_i$ будет $\sigma \rightarrow 0$, при больших значениях выходов $y_i$ будет $(1 - \sigma) \rightarrow 0$, а вместе $\sigma (1 - \sigma) \leq 0.25$ при любых аргументах)

**Функция потерь - бинарная кроссэнтропия**

$BCE(p, t) = - t \log p - (1 - t) \log (1 - p)$, где $t \in \{0,1\}, p=\sigma(y) \in (0, 1)$
- например, $t=1, y\rightarrow -\infin$, тогда $\log 0_+ \rightarrow +\infin, BCE \rightarrow +\infin$

Производная бинарной кроссэнетропии

$$\frac {\partial BCE}{\partial y} = \frac {\partial BCE}{\partial p}\frac {\partial p}{\partial y} = \frac{t}{\sigma} \sigma' + \frac{1-t}{1-\sigma} \sigma' = -t (1 - \sigma) + (1-t)\sigma = \sigma - t $$
- помним, что $p=\sigma(y)$
- видим, что "плохих" сомножителей нет
- бинарная кросс-энтропия - это функция потерь, которой хорошо оценивать вероятности и любые значения, которые находятся в интервале от 0 до 1

# Многоклассовая классификация? Софтмакс!

В этом случае выход нейросети - это полносвязный линейный слой с числом выходов по количеству классов. По сути, это по-прежнему поиск оптимальной линейной комбинации функций активации, только теперь таких комбинаций ищется несколько, и они взаимосвязаны.

**Выходная "функция активации"** - сфотмакс:
$$SM_i(\vec y) = \frac {e^{y_i}}{\sum_{j=1}^N e^{y_j}}$$
- $0 < SM_i < 1, \sum SM_i = 1$
- softmax (по-русски это - **нормализованная экспоненциальная функция**) — это обобщение логистической функции для многомерного случая
- экспонента тут нужна, чтобы нормировались также и отрицательные значения аргументов, наравне с положительными (отображение $\R\rightarrow (0,1)$), а также - потому что хорошие производные
- не может обратиться в 0, не может обратиться в 1 (кроме вырожденного случая с 1 классом)
- это на самом деле не функция активации в общепринятом смысле (преобразует выходы 1 нейрона), а преобразование пачки выходов (можно, конечно, выход тоже считать таким особо хитрым нейроном сети и тогда ОК), а вообще - это такой обоснованный мат.трюк 

Производная софтмакса ($(u/v)' = (u'v - v'u)/v^2$)
- по выходам "остальных классов"

$$\frac {\partial SM_i}{\partial y_c} = \frac{-e^{y_i}e^{y_c}}{\left(\sum_{j=1}^N e^{y_j} \right) ^2} = -SM_i SM_c$$

- по выходу данного класса

$$\frac {\partial SM_i}{\partial y_i} = \frac{-e^{y_i} \sum_{j=1}^N e^{y_j} - e^{y_i} e^{y_i}}{\left(\sum_{j=1}^N e^{y_j} \right) ^2} = SM_i (1 - SM_i)$$

**Функция потерь - кроссэнтропия**

$$CE(p, t) = - \sum_{c=1}^N t_c \log p_c$$
- где $t_c \in \{0,1\}, p=SM_c \in (0, 1)$

Производная кроссэнтропии

$$\frac {\partial CE}{\partial y_i} = - \frac{t_i}{p_i} \frac {\partial SM_i}{\partial y_i} - \sum_{c \neq i} \frac{t_c}{p_c} \frac{\partial SM_c}{\partial y_i} = - \frac{t_i}{p_i} SM_i (1 - SM_i) - \sum_{c \neq i} \frac{t_c}{p_c} (-SM_c SM_i) \Rightarrow \\
\frac {\partial CE}{\partial y_i} = - t_i (1 - p_i) + \sum_{c \neq i} t_c p_i \Rightarrow \\ 
\frac {\partial CE}{\partial y_i} = - t_i + p_i$$

- производная по выходу сети - линейна
- обращается в 0 только тогда, когда $p_i = t_i$
- $p_i$ - уверенность, что на вход был подан класс $i$
- $(1 - p_i)$ - суммарная уверенность во всех классах $c \neq i$
- просто красиво

# Локализация, детекция, сегментация и super-resolution


### Задача детекции и локализации

Обычно решаются совместно, но можно и по отдельности. Сочетает в себе решение задачи классификации и задачи восстановления скрытой зависимости (регрессии).

- У нас есть картинка, и на этой картинке где-то есть объект (возможно)
- Нам нужно на картинке найти, где этот объект находится

1. **Детекция**. Начнём с того, что нам нужно построить какую-то сеть, которая будет принимать на вход картинку и будет выдавать следующее. Для начала нам нужно определиться с тем, есть или нет на этой картинке объект. Соответственно, один из выходов нейронной сети нам должен говорить вероятность: есть здесь объект, или нет, то есть, с какой вероятностью на этой картинке находится объект. Если объекта нет, то вероятность должна быть очень маленькая, а когда объект есть, вероятность должна стремиться к 1. 

2. **Локализация**. Кроме этого, мы можем по-разному параметризовать прямоугольник, который обозначает регион, где находится объект. Я предлагаю этот прямоугольник параметризовать следующим образом: во-первых, мы должны определить, где у этого объекта находится центр это вот этот вот жёлтый крестик. И также мы должны определить размеры объекта: его высоту и ширину.

Соответственно, нейронная сеть должна нам выдавать 5 выходов:
- $p$ - вероятность наличия объекта
- $x_c, y_c$ - 2 координаты центра объекта на картинке
- $w, h$ - ширина и высота объекта на картинке

Введем **ограничения**:
- на картинке должен быть один объект или ни одного
  - чтобы не решать отдельную задачу, какой из объектов выбрать
- если центр объекта лежит в пределах картинки, то считаем, что объект есть, иначе - объекта нет
  - если видно только ухо кота, то считаем, что кота на картинке нет
- объект может выходить за пределы картинки
  - тогда выдаются ширина/высота, какие они должны быть, а не размер картинки

\*) Допустим, мы хотим локализовать (**распознать координаты**) объекта в кадре, используя **сигмоидную функцию активации**. Почему это не самый удачный вариант ФА? Эта функция очень насыщена ближе к 0 и к 1, в том смысле, что переход от y=0 к y=1 происходит в узком диапазоне, в большей части диапазона сигмоида равна либо 0 (слева), либо 1 (справа). Это приведёт к хорошему пространственному разрешению в области смены значения, но плохому разрешению по краям картинки... фактически здесь мы используем сигмоиду не по назначению, ожидая, что она будет вести себя как линейная функция. **Здесь (в задаче локализации) лучше просто использовать, например, линейную функцию активации, раз нам нужна проекция в непрерывное пространство координат**.

**Выходные функции активации**

В части детекции используется сигмодиа 
- $p = \sigma(y_0)$ - интерпретируется как вероятность наличия объекта.

В части локализации:
- координаты центра
  - сигмоиды - считаем нижний левый угол кадра (0,0), правый верхний (1,1) в масштабе этих ФА
    - $x_c = \sigma(y_1), y_c = \sigma(y_2)$
- ширина и высота
  - можно тоже сигмоиды - но так не делают (высота/ширина снизу ограничены 0, а сверху по-идее они не ограничены)
  - хороший вариант - **экспонента**:
    -  $w = \exp(y_3), h = \exp(y_4)$

**Функция потерь**

Выходы гетерогенные, функции потерь - тоже

В части детекции используется (бинарная) кроссэнтропия 
- $p: BCE(\sigma(y_0), \tilde t)$ 

В части локализации:
- координаты центра
  - $x_c, y_c: BCE(\sigma(y_1), \tilde x_c), BCE(\sigma(y_2), \tilde y_c)$
- ширина и высота
  - можно средний квадрат ошибки, например, так: 
    - $w, h: MSE(y_3, \log w), MSE(y_4, \log h)$

Итого:

$$L = BCE(\sigma(y_0), \tilde t) + \tilde t \left(BCE(\sigma(y_1), \tilde x_c) + BCE(\sigma(y_2), \tilde y_c) + MSE(y_3, \log w), MSE(y_4, \log h)\right)$$

- где $\tilde t$ также используется как индикатор для учета ошибки локализации объекта

Если сеть предсказала ($p, xc, yc, w, h$), и $p > 0$, а на самом деле объекта на картинке нет, то
- Значения функции потерь будут такими же, как если бы истинным значением было ($p=0, xc, yc, w, h$)
- Мы штрафуем нейросеть только за ошибочную классификацию ($p$), не учитывая предсказанные координаты

### Задача сегментации

Допустим, у нас есть картинка, у нас есть на ней объект, и мы хотим отделить те пиксели, на которых объект есть, от тех пикселей, на которых объекта нет.

Мы берём кадр, пропускаем через сеть - что эта сеть должна нам выдавать? 

Предлагается следующий вариант: что эта сеть нам выдаёт кадр, которая состоит из "скоров" (score) того, что в каждом из пикселей есть или нет объект. Допустим, мы сделали такую сеть, которая выдаёт кадр, там есть "скоры" для каждого пикселя. Вопрос: как теперь сделать функцию потерь для этой задачи? **При этом для обучения у нас есть правильная сегментация объекта**.

Получается **задача бинарной сегментации** - отделение объекта от фона

**Выходная фукнция активации** 
- сигмоида - дает для каждого пикселя "вероятность" отнесения к объекту

**Функция потерь**
- суммарная бинарная кроссэнтропия по всем пискелям кадра

$$L = \sum_j BCE(\sigma(y_j), t_j)$$

### Задача сжатия размерности

Есть кадр, в котором содержится большое количество разной информации. 

Мы хотим сделать сеть **(авто-)энкодер**, которая по кадру выдет **векторное представление информации**, содержащейся в кадре - **эмбеддинг**, и на котором другая сеть **декодер** делает некоторую расшифровку - переводит это векторное представление обратно в изображение.

Внутренне векторное представление (эмбеддинг) будет меньшей размерности, чем исходный кадр, иначе смысла нет.

**Функция потерь**
- можно просто сравнивать входной и выходной кадр (например, попиксельный MSE)
$$L = \sum_i MSE(Img_i, \tilde {Img_i}$$
- можно перевести пиксели кадра в [0,1], на выход декодера - софтмакс, и использовать бинарную кроссэнтропию между входным и выходным кадром

$$L = \sum_i BCE(Img_i, \sigma (y_i))$$

В данном случае **целью обучения** сети является получение векторных представлений кадров (**получение эмбеддингов**), которые можно потом с хорошей эффективностью использовать в других задачах
- например, сжимать кадр таким обученным автоэнкодером и на полученном эмбеддинге уже тренировать другую сеть решать задачу классификации объектов в кадре (или вообще без сети использовать - деревья решений, линейная регрессия и т.п.)

### Задача super-resolution

Задача "super resolution" заключается в следующем: у нас есть какая-то картинка, с очень низким разрешением. Мы хотим это разрешение увеличить в 2 раза по каждому измерению, то есть по X и по Y. Как сделать такую нейронную сеть?

Cеть принимает на вход картинку и на каждый пиксель входной картинки выдаёт 4 пикселя выходной картинки. 

Cеть можно обучать примерно так: исходный набор кадров высокого разрешения сжимается в 2 раза по каждому измерению, подается на вход и сравнивается выход и исходные кадры до сжатия.

В отличие от алгоритмической пикселизации/интерполяции здесь нейросеть может дорисовать детали, которых на сжатом изображении нет, то которые были на обучающих кадрах (шерстинки кота, детали лица, текстуры одежды и т.п.)

### Другие задачи CV

Стоит отметить, что класс задач, которые решаются при помощи нейронных сетей очень велик. И он далеко не ограничивается теми задачами, которые мы рассмотрели. Каждый год появляются новые задачи, нередко эти задачи попадают в класс "классификации" некоторые из этих задач попадают в разряд "регрессии". Но бывают такие, которые совмещают в себе и классификацию, и регрессию, и очень часто они бывают довольно хитрыми, и мы с вами уже разобрали, каким образом можно решать такие задачи: нужно придумывать функции активации и придумывать правильные функции потерь для того, чтобы решать эти задачи.

# Теоретические задачи: Функции потерь

Пусть имеется монетка, которую мы подбрасывали $N$ раз, и $M$ раз монетка выпала орлом вверх. Мы будем восстанавливать вероятность выпадения **орла** $p$ при помощи минимизации бинарной кросс-энтропии:

$$\tilde{p} = \arg\min_p \sum_{i=1}^N \left( - t_i \log p - (1 - t_i) \log(1 - p) \right),$$

где $\arg\min_x f(x)$ - значение $x$, при котором $f$ минимальна, $t_i = 1$ в том случае, когда выпал орел и $t_i = 0$ в том случае, когда выпала решка.

Найдите $\tilde{p}$ и запишите это выражение в качестве ответа (синтаксис формул: python/sympy).

$p = - M \log p - (N-M) \log(1 - p)$

$p' = -M / p - (N-M) / (1 - p) = 0$

$p_0 = M/N$


In [12]:
import sympy as sp

p, N, M = sp.symbols("p N M")

L = - M * sp.log(p) - (N - M) * sp.log(1 - p)
dL = L.diff(p)
p0 = sp.solve(dL, p)

sp.Set("dL =", dL, "  p0 =", p0)

Set(dL =, -M/p - (M - N)/(1 - p),   p0 =, [M/N])

Допустим, имеется много наблюдений (при абсолютно одинаковых параметрах, - вообще не при чем тут): $\{ y_i \}_{i=1}^N$. Сформулируем задачу восстановления значения $\mu$ следующим образом:

$\mu = \arg\min_{\tilde{y}} \sum_{i=1}^N(\tilde{y} - y_i)^2$

Найдите, чему будет равно $\mu$ (т.е. $\tilde{y}$ при котором выражение минимально):

$$\frac{\partial L}{\partial \tilde y} = 2\sum_{i=1}^N(\tilde{y} - y_i) = 2 N \tilde{y} - 2 \sum_{i=1}^N y_i = 0 \\
\tilde{y} = \left(\sum_{i=1}^N y_i \right) / N $$

Допустим, имеется много наблюдений при абсолютно одинаковых параметрах: $\{ y_i \}_{i=1}^{2N - 1}$. Сформулируем задачу восстановления значения $\mu$ следующим образом:

$\mu = \arg\min_{\tilde{y}} \sum_{i=1}^{2N - 1} |\tilde{y} - y_i|$

Найдите, чему будет равно $\tilde{y}$, минимизирующее функцию потерь, если все значения $y_i$ расположены по возрастанию. 

- можно нетривиально ковырять тривиальную производную, но тут все в уме - это такое значение, которое будет в середине упорядоченного ряда наблюдений, остальные в производной сокращаются

$$\tilde y = y_N$$

Пусть имеется выборка $\{ t_i \}_{i=1}^N$ некоторых чисел $t_i \in (0, 1)$

Запишите решение оптимизационной задачи:

$\mu = \arg\min_{p} \sum_{i=1}^N \left(- t_i \log p - (1 - t_i) \log (1 - p)\right)$

$$\frac{\partial L}{\partial p} = \sum_{i=1}^N \left(- \frac{t_i}{p} - \frac{1 - t_i} {1 - p} (-1)\right) = - \frac{1}{p}\sum_{i=1}^N t_i + \frac{1} {1 - p} \sum_{i=1}^N (1 - t_i) = \\

= - \frac{1}{p}\sum_{i=1}^N t_i + \frac{N} {1 - p} - \frac{1} {1 - p}\sum_{i=1}^N t_i = \frac{N} {1 - p} - \frac{p + 1 - p}{p(1-p)}\sum_{i=1}^N t_i = 0\\

\frac{N} {1 - p} = \frac{\sum_{i=1}^N t_i} {p(1 - p)} \\


p = \frac{\sum_{i=1}^N t_i} {N} $$

Опять метод максимального правдоподобия и матожидание как оценка параметра

**Семинары** по регрессии (восстановление неизвестной зависимости, аппроксимация) и классификации в `./Neural_Networks_and_CV`